## THIS NOTEBOOK IS NOT USED

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.16.1'

In [3]:
batch_size = 128
img_height = 48
img_width = 48

In [4]:
resnet101 = tf.keras.applications.ResNet101V2(
    input_shape=(img_height, img_width, 3),
    pooling='avg',
    include_top=False
)

In [5]:
len(resnet101.layers)

378

In [5]:
import pathlib
data_dir_train = pathlib.Path('./images/images/train')
data_dir_val = pathlib.Path('./images/images/train')

Load the data as RGB just to match the channels

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  )

Found 28821 files belonging to 7 classes.


In [7]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  label_mode='categorical',
  batch_size=batch_size,
  )

Found 28821 files belonging to 7 classes.


In [8]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input

def custom_preprocess_input(image, label):
    image = preprocess_input(image)
    return image, label

In [9]:
new_train_ds = train_ds.map(custom_preprocess_input)

In [10]:
new_val_ds = val_ds.map(custom_preprocess_input)

In [8]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import RandomFlip, RandomRotation, Rescaling, RandomZoom

# rescaling = Sequential([
#     Rescaling(1./127.5, offset=-1),
# ])

# preprocessing_seq = Sequential([
#     rescaling,
#     RandomFlip(),
#     RandomRotation(0.2),
#     RandomZoom(
#         height_factor=(-0.2, 0.2),
#         width_factor=(-0.2, 0.2)
#     )
# ])

In [9]:
# new_train_ds = train_ds.map(
#     lambda x, y: (preprocessing_seq(x, training=True), y)
# )

# new_val_ds = val_ds.map(
#     lambda x, y: (rescaling(x), y)
# )

In [11]:
num_classes = 7

In [ ]:
resnet101.summary()

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, BatchNormalization, Dropout, Flatten

repurposed = Sequential([
    resnet101, 
    Flatten(),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax'),
])

In [13]:
resnet101.trainable = False

In [17]:
for layer in resnet101.layers[:341]:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy

repurposed.compile(optimizer=Adam(learning_rate=0.01), loss=categorical_crossentropy, metrics=['accuracy', AUC()])

In [15]:
repurposed.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101v2 (Functional)        │ ?                      │    42,626,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,626,560 (162.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 42,626,560 (162.61 MB)

In [16]:
resnet101.trainable

False

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

epochs = 50

reduce = ReduceLROnPlateau(monitor = 'val_loss', patience = 4, verbose = 1, factor = 0.50, min_lr = 1e-6)

repurposed.fit(new_train_ds, validation_data=new_val_ds, epochs=epochs, callbacks=[reduce])

Epoch 1/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 150s 624ms/step - accuracy: 0.2627 - auc: 0.6596 - loss: 1.8559 - val_accuracy: 0.3869 - val_auc: 0.7683 - val_loss: 1.5784 - learning_rate: 0.0100
Epoch 2/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 142s 628ms/step - accuracy: 0.3613 - auc: 0.7518 - loss: 1.6272 - val_accuracy: 0.4340 - val_auc: 0.8048 - val_loss: 1.4745 - learning_rate: 0.0100
Epoch 3/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 138s 609ms/step - accuracy: 0.4002 - auc: 0.7778 - loss: 1.5597 - val_accuracy: 0.4370 - val_auc: 0.8036 - val_loss: 1.4873 - learning_rate: 0.0100
Epoch 4/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 141s 622ms/step - accuracy: 0.4170 - auc: 0.7906 - loss: 1.5218 - val_accuracy: 0.3955 - val_auc: 0.7872 - val_loss: 1.5700 - learning_rate: 0.0100
Epoch 5/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 137s 605ms/step - accuracy: 0.4348 - auc: 0.8023 - loss: 1.4873 - val_accuracy: 0.4733 - val_auc: 0.8213 - val_loss: 1.4117 - learning_rate: 0.0100
Epoch 6/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 144s 638ms/step - ac

In [19]:
repurposed.save('resnetfinedtuned.keras')

In [34]:
start_tuning = 274

for layer in resnet101.layers[start_tuning:]:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True

In [35]:
for i, layer in enumerate(resnet101.layers):
    print(i, layer.name, "-", layer.trainable)

0 input_layer - False
1 conv1_pad - False
2 conv1_conv - False
3 pool1_pad - False
4 pool1_pool - False
5 conv2_block1_preact_bn - False
6 conv2_block1_preact_relu - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_pad - False
11 conv2_block1_2_conv - False
12 conv2_block1_2_bn - False
13 conv2_block1_2_relu - False
14 conv2_block1_0_conv - False
15 conv2_block1_3_conv - False
16 conv2_block1_out - False
17 conv2_block2_preact_bn - False
18 conv2_block2_preact_relu - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_pad - False
23 conv2_block2_2_conv - False
24 conv2_block2_2_bn - False
25 conv2_block2_2_relu - False
26 conv2_block2_3_conv - False
27 conv2_block2_out - False
28 conv2_block3_preact_bn - False
29 conv2_block3_preact_relu - False
30 conv2_block3_1_conv - False
31 conv2_block3_1_bn - False
32 conv2_block3_1_relu - False
33 conv2_block3_2_pad - Fal

In [36]:
from tensorflow.keras.optimizers import RMSprop

repurposed.compile(optimizer=RMSprop(0.00001), loss=categorical_crossentropy, metrics=['accuracy', AUC()])

In [37]:
repurposed.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101v2 (Functional)        │ (None, 2048)           │    42,626,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,204,551 (164.81 MB)

 Trainable params: 22,214,215 (84.74 MB)

 Non-trainable params: 20,990,336 (80.07 MB)

In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 20

reduce = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.50, min_lr = 1e-7)
checkpoint = ModelCheckpoint(filepath='resnetfinedtunedv2.keras', save_best_only=True, verbose=1, monitor='val_accuracy', mode='max')

repurposed.fit(new_train_ds, validation_data=new_val_ds, epochs=epochs, callbacks=[reduce, checkpoint])

Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 680ms/step - accuracy: 0.7281 - auc_2: 0.9453 - loss: 0.8052
Epoch 1: val_accuracy improved from -inf to 0.83071, saving model to resnetfinedtunedv2.keras
226/226 ━━━━━━━━━━━━━━━━━━━━ 254s 1s/step - accuracy: 0.7281 - auc_2: 0.9453 - loss: 0.8052 - val_accuracy: 0.8307 - val_auc_2: 0.9763 - val_loss: 1.0752 - learning_rate: 1.0000e-05
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.7353 - auc_2: 0.9479 - loss: 0.7827
Epoch 2: val_accuracy improved from 0.83071 to 0.85226, saving model to resnetfinedtunedv2.keras
226/226 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.7353 - auc_2: 0.9479 - loss: 0.7826 - val_accuracy: 0.8523 - val_auc_2: 0.9800 - val_loss: 0.9710 - learning_rate: 1.0000e-05
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.7405 - auc_2: 0.9506 - loss: 0.7629
Epoch 3: val_accuracy did not improve from 0.85226
226/226 ━━━━━━━━━━━━━━━━━━━━ 219s 969ms/step - accuracy: 0.7406 - auc_2: 0.9506 - 